# ビジネスへの架け橋となる機械学習モデルの評価指標

評価指標は機械学習モデルを評価するためにあるが、ビジネス、つまりKPIへ直接的に影響を与えるものではないことが多い。  
本来機械学習はビジネス上の目標(KGIやKPI)を達成するための手段である。  
せっかく精度の良いモデルを作っても、ビジネス的視点においては無駄になってしまうこともある。  
そこで、モデルの良さを表す評価指標をKPIへ繋ぐ、またKPIへ影響を与える評価指標を選ぶという視点で、評価指標についてまとめる。

## ビジネスインパクトによる閾値調整
二値分類モデルにおける通常運用では、閾値を0.5に設定してPositive・Negativeを予測していることが多いと思う。  
評価指標自体は分類の正しさに注目しており、ビジネスモデル固有の要素を反映していない。つまり評価指標だけを見てビジネスに適用した時に寄与しづらいことがある。  
一方で、ビジネスインパクトの期待値を計算しておくと、ビジネス的視点からモデルの評価もできるので、寄与しやすくなる。  
本項においては、ビジネスへのインパクトを第一と考えた時に、そのインパクトを最大化する手段の一つである閾値調整を見ていく。
施策依存なので考え方は様々だと思うが、ここでは題材を用意して、具体的なコードを書いていきたい。

### 題材
- dataset : https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets
- 年齢、職種、教育水準、電話によって定期預金を契約したかなどのカラムがある。
- 概要 : 定期預金を契約する可能性が最も高い顧客を特定するというタスク
- 条件 : コールセンターの人件費がワンコールあたり200円、一人契約の利益は10000円、機会損失は含めない
- 今回はビジネスインパクトの期待値を利用した閾値の調整方法として、**利益曲線**を描画する。

- コスト行列は以下のようになる
    - True Positive : モデルが契約すると予測し、実際に契約したユーザ。10000円-200円=9800円の利益。
    - False Positive : モデルが契約すると予測し、契約しなかったユーザ。200円の損失。
    - True Negative : モデルが契約しないと予測し、契約しなかったユーザ。0円。
    - False Negative : モデルが契約しないと予測し、契約したユーザ。機会損失となるが、今回は含めない。(ここは現場による。)

In [9]:
# ライブラリのインポート
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB

In [10]:
df = pd.read_csv('data/train.csv', sep = ';')
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [14]:
# 期待値を計算するための関数
def calc_expected_value(conf_matrix):
    # コスト行列
    TP_COST, FP_COST, TN_COST, FN_COST = 9800, -200, 0, 0
    tp, fn, fp, tn = conf_matrix.flatten()

    # 各項目の出現確率を算出
    number_of_sample = np.sum(conf_matrix)
    tp_rate = tp/ number_of_sample
    fp_rate = fp / number_of_sample
    tn_rate = tn / number_of_sample
    fn_rate = fn / number_of_sample
    # 利益の期待値を計算
    return (tp_rate * TP_COST) + (fp_rate * FP_COST) + (tn_rate * TN_COST) + (fp_rate * FP_COST)

In [15]:
def benefit_curve(clf_list, X_val, y_val):
    # 分類器ごとに利益曲線を描画する
    for clf in clf_list:
        y_val_hat_proba = clf.predict_proba(X_val)

        expected_value_list = []
        thresh_list = []
        for proba in np.sort(y_val_hat_proba[:, 1]):
            # 予測したラベル
            y_val_hat = (y_val_hat_proba[:, 1] > proba).astype(int)
            # 混同行列を作成する
            conf_matrix = confusion_matrix(y_val, y_val_hat, labels = [1, 0])
            # 期待値を計算
            expected_value = calc_expected_value(conf_matrix)
            expected_value_list.append(expected_value)
            thresh_list.append(proba)
        # コストが最大の時のインデックス番号を取得する
        max_cost_index = expected_value_list.index(max(expected_value_list))
        plt.plot(thresh_list, expected_value_list, label = f"{clf.__class__.__name__}")
        print(f"{clf.__class__.__name__}を使った時に利益が最大になる閾値：{thresh_list[max_cost_index]}")
        plt.xlabel("threshold")
        plt.ylabel("expected value")
        plt.legend(bbox_to_anchor = (1, 1), loc = 'upper right', borderaxespad = 0, fontsize = 10)
        plt.show()